In [93]:
import pandas as pd
import math
from collections import defaultdict

In [94]:
office_df = pd.read_csv("OfficeSpace.csv")
office_df.rename(columns={"Employee Name" : "Employee", "Manager Name" : "Manager"}, inplace=True)
office_df


,Employee,Manager
0,Bill Lumbergh,NaN
1,Bob Slydell,Bill Lumbergh
2,Bob Porter,Bill Lumbergh
3,Linda M. Grayson,Bill Lumbergh
4,Dom Portwood,Linda M. Grayson
5,Wendy L. Hargrove,Linda M. Grayson
6,Tom Smykowski,Linda M. Grayson
7,Nathan R. Ross,Linda M. Grayson
8,Peter Gibbons,Dom Portwood
9,Cheryl T. Ackerman,Peter Gibbons


In [95]:
office_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Employee  25 non-null     object
 1   Manager   24 non-null     object
dtypes: object(2)
memory usage: 532.0+ bytes


In [96]:
employee_to_managers = dict()
manager_to_employee = defaultdict(list)

for s in office_df.itertuples():
    manager = s.Manager
    employee = s.Employee
    if isinstance(manager, float) and math.isnan(manager):
        employee_to_managers[employee] = None 
    else:
        employee_to_managers[employee] = manager
        manager_to_employee[manager].append(employee)

employee_to_managers
print(manager_to_employee)

defaultdict(<class 'list'>, {'Bill Lumbergh': ['Bob Slydell', 'Bob Porter', 'Linda M. Grayson'], 'Linda M. Grayson': ['Dom Portwood', 'Wendy L. Hargrove', 'Tom Smykowski', 'Nathan R. Ross'], 'Dom Portwood': ['Peter Gibbons', 'Samir Nagheenanajar', 'Nathan C. Carter'], 'Peter Gibbons': ['Cheryl T. Ackerman'], 'Samir Nagheenanajar': ['Michael Bolton'], 'Nathan C. Carter': ['Lydia Bennett', 'Greg S. Torres', 'Alice L. Munroe'], 'Wendy L. Hargrove': ['Peggy Carlson', 'Anne Martinez', 'Fred Wilkinson'], 'Tom Smykowski': ['Derek P. Phillips', 'Milton Waddams'], 'Derek P. Phillips': ['Sarah J. Greene'], 'Nathan R. Ross': ['Alan B. Peterson'], 'Alan B. Peterson': ['Maria D. Sanchez', 'Bobbie K. Jenkins']})


In [97]:
office_df['Direct Reports'] = office_df["Employee"].apply(lambda x: len(manager_to_employee[x]))
office_df

,Employee,Manager,Direct Reports
0,Bill Lumbergh,NaN,3
1,Bob Slydell,Bill Lumbergh,0
2,Bob Porter,Bill Lumbergh,0
3,Linda M. Grayson,Bill Lumbergh,4
4,Dom Portwood,Linda M. Grayson,3
5,Wendy L. Hargrove,Linda M. Grayson,3
6,Tom Smykowski,Linda M. Grayson,2
7,Nathan R. Ross,Linda M. Grayson,1
8,Peter Gibbons,Dom Portwood,1
9,Cheryl T. Ackerman,Peter Gibbons,0


In [98]:
def get_total_reports(employee: str, manager_to_employee: dict):
    if len(manager_to_employee[employee]) == 0:
        return 0
    total = len(manager_to_employee[employee])
    for reporter in manager_to_employee[employee]:
        total += get_total_reports(reporter, manager_to_employee)
    return total

# Apply it to the DataFrame
office_df["Total Reports"] = office_df.apply(
    lambda x: get_total_reports(x["Employee"], manager_to_employee),
    axis=1
)

office_df


,Employee,Manager,Direct Reports,Total Reports
0,Bill Lumbergh,NaN,3,24
1,Bob Slydell,Bill Lumbergh,0,0
2,Bob Porter,Bill Lumbergh,0,0
3,Linda M. Grayson,Bill Lumbergh,4,21
4,Dom Portwood,Linda M. Grayson,3,8
5,Wendy L. Hargrove,Linda M. Grayson,3,3
6,Tom Smykowski,Linda M. Grayson,2,3
7,Nathan R. Ross,Linda M. Grayson,1,3
8,Peter Gibbons,Dom Portwood,1,1
9,Cheryl T. Ackerman,Peter Gibbons,0,0


In [99]:
def hierarchy_data(employee_to_managers : dict, employee):
    manager = employee_to_managers.get(employee)
    if manager is None:
        return employee
    ans = []
    ans.append(employee)
    while manager is not None:
        ans.append(manager)
        manager = employee_to_managers[manager]
    ans[:] = ans[::-1]
    return " > ".join(ans)


In [100]:
office_df["Reporting Hierarchy"] = office_df.apply(lambda x: hierarchy_data(employee_to_managers, x.Employee), axis=1)
office_df

,Employee,Manager,Direct Reports,Total Reports,Reporting Hierarchy
0,Bill Lumbergh,NaN,3,24,Bill Lumbergh
1,Bob Slydell,Bill Lumbergh,0,0,Bill Lumbergh > Bob Slydell
2,Bob Porter,Bill Lumbergh,0,0,Bill Lumbergh > Bob Porter
3,Linda M. Grayson,Bill Lumbergh,4,21,Bill Lumbergh > Linda M. Grayson
4,Dom Portwood,Linda M. Grayson,3,8,Bill Lumbergh > Linda M. Grayson > Dom Portwood
5,Wendy L. Hargrove,Linda M. Grayson,3,3,Bill Lumbergh > Linda M. Grayson > Wendy L. Ha...
6,Tom Smykowski,Linda M. Grayson,2,3,Bill Lumbergh > Linda M. Grayson > Tom Smykowski
7,Nathan R. Ross,Linda M. Grayson,1,3,Bill Lumbergh > Linda M. Grayson > Nathan R. Ross
8,Peter Gibbons,Dom Portwood,1,1,Bill Lumbergh > Linda M. Grayson > Dom Portwoo...
9,Cheryl T. Ackerman,Peter Gibbons,0,0,Bill Lumbergh > Linda M. Grayson > Dom Portwoo...


In [101]:
total_reports = office_df["Total Reports"].sum().sum()
total_reports

70